In [ ]:
import os
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16)


# model_name = "/home/jupyter/2581954llmaccess/me-llama/70bchat/" 
model_name = "/home/jupyter/2581954llmaccess/huggingface/models--meta-llama--Llama-3.1-70B-Instruct/snapshots/945c8663693130f8be2ee66210e062158b2a9693/"

import torch
# load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_name,)
tokenizer.add_special_tokens({'pad_token': 'none'})
tokenizer.add_special_tokens({'pad_token': '<pad>'})
tokenizer.add_eos_token = True
tokenizer.pad_token_id = 0
tokenizer.padding_side = "left"

model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=quant_config, device_map='auto')

In [ ]:
question = 'QUESTRION HERE'


inputs = tokenizer(question, return_tensors="pt")

outputs = model.generate(
    **inputs,
    max_new_tokens=300,
)

generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(f"Generated text: {generated_text}")

In [ ]:
# prompts for other tasks

In [10]:
#DEFINE YOUR OWN TASK
# FOR NER TASK
prompt_1 = '''### Your task is to generate an HTML version of an input text, using HTML <span> tags to mark up specific entities.

### Entity Markup Guides:
Use <span class="Language_Fluent"> to denote a language spoken by the patient fluently.
Use <span class="Language_Some"> to denote a language spoken by the patient in moderate level.
Use <span class="Language_No"> To denote a language which cannot be spoken or can only be spoken a little by the patient.
Use <span class="Language_Other"> to denote a language that not related to patient.

### Entity Definitions:
Language_Fluent: The person speaks the language fluently, including native speakers and those who have achieved nearnative fluency. They can use the language effectively in  various contexts with complete fluency and cultural understanding. Instances where a patient’s fluency is not explicitly stated but can be directly inferred (e.g. mention of interpreter/translator, preference of language on prescription) are included in this definition.
Language_Some: The person has a moderate level of proficiency in the language. They can understand and use the language for basic communication and simple conversations but are not fully fluent.
Language_No: The person knows a few words or phrases in the language but cannot use it for basic communication. Their knowledge is very limited and not sufficient for meaningful  interaction. Or the person does not know or speak the language at all. Cannot use this Language to communicate at all.
Language_Other: Languages mentioned in the text that are not related to the person’s language proficiency. These may be languages discussed in a different context or related to other individuals. Words that could refer to a language but are not used in that context (e.g. Greek yogurt, Irish Catholic, French catheter size) should not be annotated.

### Additional Rules:
1. Only annotate the name of the language itself.
2. Do not annotate descriptive words about the language proficiency level (e.g. "some", "simple") or negations (e.g. "not", "no").
3. Hyphenated languages (ex. Chinese-Mandarin) or language connected by "/" should be labelled separately for each token.

### Input Text: {}<EOS>
### Output Text:'''


# FOR CLASSIFICATION TASK
prompt_2 = """***TASK***
The task is to decide relevant COVID-19 topics of the article based on its abstract.

***INPUT***
The input is an abstract text.

***DOCUMENTATION***
There are 7 topics you will need to decide whether the article is related to. The followings are the topics and their definitions.
Mechanism: underlying cause(s) of COVID-19 infections and transmission and possible drug mechanism of action. 
Transmission: characteristics and modes of COVID-19 transmissions. 
Diagnosis: COVID-19 assessment through symptoms, test results and radiological features for COVID-19. 
Treatment: treatment strategies, therapeutic procedures and vaccine development for COVID-19. 
Prevention: prevention, control, mitigation and management strategies for COVID-19. 
Case Report: descriptions of specific patient cases related to COVID-19. 
Epidemic Forecasting: estimation on the trend of COVID-19 spread and related modeling approach. 
Other: other situations that can not be included in above.

***OUTPUT***
The output should be in a json format, with relevant value for each topic: Treatment, Diagnosis, Prevention, Mechanism, Transmission, Epidemic Forecasting, Case Report.
Put value 1 if the article is related to that topic, 0 if the article is not related to that topic. 
Please note one article can be related to multiple topics.
Example output format:
```json
{{
  "Treatment":,
  "Diagnosis":,
  "Prevention":,
  "Mechanism":,
  "Transmission":,
  "Epidemic Forecasting":,
  "Case Report":,
  "Other":
}}

Input: {} <EOS>
Output:"""

In [11]:
question = prompt_1.format('I speak english')